In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os as os
import math

#TODO Introducir el dataset con el que se vaya a trabajar
station = 'palmer'
df = pd.read_csv(f'data_{station}/{station}_raw.csv')
cols = [col for col in df.columns]

C:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Temp\ipykernel_18516\2634773747.py:10: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'data_{station}/{station}_raw.csv')


In [5]:
# Asumiendo que 'date' está en formato datetime y 'temp' es numérica
plt.figure(figsize=(10, 6))
sns.lineplot(x='date', y='temp', data=df)

plt.xlabel('Date')  # Etiqueta para el eje X
plt.ylabel('Temperature')  # Etiqueta para el eje Y
plt.title('Temperature over Time')  # Título de la gráfica

plt.xticks(rotation=45)  # Rotar las etiquetas del eje X si es necesario
plt.tight_layout()  # Ajustar el layout para evitar solapamientos

plt.savefig(f'test_{station}.png')

plt.show()

TypeError: 'value' must be an instance of str or bytes, not a float

In [11]:

# Pasamos la columna 'date' a formato datetime

try:
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
except ValueError:  # Capturamos la excepción correcta
    for i in range(len(df)):
        fecha = df.loc[i, 'date']  

        if len(fecha) == 10:  # Verificamos si la fecha no tiene hora
            fecha += ' 00:00:00'  # Añadimos '00:00:00'
        
            # Convertimos la cadena a formato datetime
            df.loc[i, 'date'] = pd.to_datetime(fecha)


df.info()

KeyError: 'date'

In [9]:
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d %H:%M:%S', errors='coerce')

# Filtrar las filas con fechas inválidas
filas_invalidas = df[df['date'].isna()]

# # Mostrar las filas con fechas incorrectas
# print(filas_invalidas)

num_filas_invalidas = filas_invalidas.isnull().sum()
num_filas_invalidas


date         383556
vel          380676
dir          380676
WGS.10min    380676
WGD.10min    380676
temp         382051
hr           382051
dew          382877
pres         382116
prec         382116
dtype: int64

In [12]:
df.head()

,date,dir,vel,present_weather,past_weather,pres,temp,d2m
201207,1996-01-01 00:00:00,200.0,4.0,60.0,22.0,991.2,1.9,1.9
201206,1996-01-01 01:00:00,170.0,2.0,NaN,NaN,989.5,1.5,1.5
201205,1996-01-01 02:00:00,140.0,4.0,NaN,NaN,987.4,2.0,2.0
201204,1996-01-01 03:00:00,240.0,4.0,60.0,62.0,986.5,1.6,1.6
201203,1996-01-01 04:00:00,60.0,26.0,NaN,NaN,986.2,5.5,2.9


In [3]:

# En caso de que los nulos tengan un formato diferente

df.replace(444, np.nan, inplace=True)

### Calcular la humedad relativa

Se aplica la formula de magnus a partir de la temperatura y el punto de rocío.

In [14]:
# Definir la función para calcular la presión de vapor de saturación utilizando la ecuación de Magnus
def presion_vapor_saturacion(T):
    return 6.1094 * np.exp((17.625 * T) / (T + 243.04))

# Definir la función para calcular la humedad relativa
def calcular_humedad_relativa(temp_aire, temp_rocio):
    # Calcular la presión de vapor para la temperatura del aire y el punto de rocío
    e_aire = presion_vapor_saturacion(temp_aire)
    e_rocio = presion_vapor_saturacion(temp_rocio)
    
    # Calcular la humedad relativa
    humedad_relativa = 100 * (e_rocio / e_aire)
    
    return humedad_relativa

# Aplicar la función a tu DataFrame, asumiendo que 'temp' es la temperatura del aire y 'd2m' el punto de rocío
df['hr'] = calcular_humedad_relativa(df['temp'], df['d2m'])

# Mostrar las primeras filas con la nueva columna de humedad relativa
print(df[['temp', 'd2m', 'hr']].head(30))

        temp  d2m          hr
201207   1.9  1.9  100.000000
201206   1.5  1.5  100.000000
201205   2.0  2.0  100.000000
201204   1.6  1.6  100.000000
201203   5.5  2.9   83.343240
201202   5.4  2.0   78.722874
201201   3.7  1.5   85.540069
201200   3.6  1.6   86.763330
201199   3.4  1.5   87.367093
201198   3.5  1.6   87.376700
201197   3.5  1.6   87.376700
201196   3.8  1.6   85.550941
201195   3.5  1.5   86.753294
201194   3.7  1.5   85.540069
201193   3.1  1.6   89.878996
201192   2.7  1.9   94.465770
201191   2.8  1.6   91.808170
201190   3.4  1.5   87.367093
201189   3.7  1.4   84.929269
201188   2.4  1.8   95.813257
201187   2.2  1.9   97.883395
201186   2.5  2.0   96.502879
201185   2.7  2.1   95.823269
201184   2.7  1.6   92.461451
201183   2.2  1.5   95.123011
201182   1.0  1.0  100.000000
201181   1.2  1.2  100.000000
201180   0.8  0.8  100.000000
201179   1.3  1.3  100.000000
201178   0.5  0.5  100.000000


In [11]:
cols

['date',
 'dir',
 'vel',
 'present_weather',
 'past_weather',
 'pres',
 'temp',
 'd2m']

### Uniformamos el dataset

-Eliminamos las columnas que no nos interesen

-Fijamos la columan de fecha

In [15]:
# date,temp,skt,dir,vel,hr,prec,pres

columns_to_drop = [
 'present_weather',
 'past_weather',
  'd2m'
    ]

df_2 = df.drop(columns=columns_to_drop)

df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201208 entries, 201207 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    201208 non-null  datetime64[ns]
 1   dir     201192 non-null  float64       
 2   vel     196441 non-null  float64       
 3   pres    201166 non-null  float64       
 4   temp    201125 non-null  float64       
 5   hr      122123 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 10.7 MB


In [29]:
df_2['date'] = df_2['date'].apply(lambda x: x if ':' in str(x) else str(x) + ' 00:00:00')
df_2['date'].head()

df_2['date'] = pd.to_datetime(df_2['date'])

In [16]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201208 entries, 201207 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    201208 non-null  datetime64[ns]
 1   dir     201192 non-null  float64       
 2   vel     196441 non-null  float64       
 3   pres    201166 non-null  float64       
 4   temp    201125 non-null  float64       
 5   hr      122123 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 10.7 MB


In [17]:
#Recuento de nulos

df_2.isnull().sum()

date        0
dir        16
vel      4767
pres       42
temp       83
hr      79085
dtype: int64

In [18]:
# Recuento de fechas duplicadas
duplicates = df_2.duplicated(subset='date', keep='first').sum()

duplicates

0

### Reindexación de las columnas

In [19]:
cols = [col for col in df_2.columns]
cols

['date', 'dir', 'vel', 'pres', 'temp', 'hr']

In [20]:
# date,temp,skt,dir,vel,hr,prec,pres
new_order = ['date', 'temp', 'dir', 'vel', 'hr',  'pres']
df = df.reindex(columns=new_order)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201208 entries, 201207 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    201208 non-null  datetime64[ns]
 1   temp    201125 non-null  float64       
 2   dir     201192 non-null  float64       
 3   vel     196441 non-null  float64       
 4   hr      122123 non-null  float64       
 5   pres    201166 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 10.7 MB


In [21]:
df_2.to_csv(f'data_{station}/{station}_clean.csv', index=False, columns=new_order)

### Visualizacion de los datos

In [1]:


num_cols = len(df_2.columns) - 1  
num_rows = math.ceil(num_cols / 4)  


fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(20, 5 * num_rows))  

# Aplanar el array de ejes para poder iterar sobre él
axes = axes.flatten()

# Iterar sobre las columnas (excluyendo 'fhora') y generar gráficos
for ax, col in zip(axes, df_2.columns.drop(date)):
    print(col, '--->' ,type(df_2[col]))
    df_2[col] = pd.to_numeric(df_2[col], errors='coerce')
    ax.plot(df_2['date'], df_2[col])
    ax.set_xlabel('date')
    ax.set_ylabel(col)
    ax.set_title(f'{col} a lo largo del tiempo')

# Eliminar los ejes sobrantes si el número de columnas no es múltiplo de 4
if num_cols % 4:
    for ax in axes[num_cols:]:
        ax.remove()

plt.savefig(f'resource/clean_{station}.png')

plt.tight_layout()
plt.show()

NameError: name 'df_2' is not defined